<a href="https://colab.research.google.com/github/megmarv/Client-Deposit-Prediction/blob/main/MachineLearningCoursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git status


fatal: not a git repository (or any of the parent directories): .git


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

# Load the dataset
data_path = '/content/bank-additional-full.csv'
df = pd.read_csv(data_path, sep=';')

# Display initial information
print("Initial Dataset Info:")
df.info()
print("\nSample Data:")
print(df.head())

# 1. Handle missing values ("unknown")
missing_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan']
for col in missing_cols:
    mode_value = df[col].mode()[0]
    df[col].replace('unknown', mode_value, inplace=True)

# 2. Remove duplicate rows
df.drop_duplicates(inplace=True)

# 3. Drop or exclude the `duration` column
df.drop(columns=['duration'], inplace=True)

# 4. One-hot encode categorical variables and scale numeric features
categorical_columns = [
    'job', 'marital', 'education', 'default', 'housing', 'loan',
    'contact', 'month', 'day_of_week', 'poutcome'
]
numeric_columns = [
    'age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
    'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'
]

# Transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder(drop='first'), categorical_columns)
    ]
)

X = df.drop(columns=['y'])
y = df['y'].apply(lambda x: 1 if x == 'yes' else 0)  # Convert target to binary (0, 1)

# Apply preprocessing
X_preprocessed = preprocessor.fit_transform(X)

# 5. Handle class imbalance
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X_preprocessed, y)

# 6. Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# Save processed data for modeling
pd.DataFrame(X_train).to_csv('/content/X_train.csv', index=False)
pd.DataFrame(X_test).to_csv('/content/X_test.csv', index=False)
pd.DataFrame(y_train).to_csv('/content/y_train.csv', index=False)
pd.DataFrame(y_test).to_csv('/content/y_test.csv', index=False)

print("Preprocessing complete. Datasets saved for modeling.")


KeyError: "['balance'] not in index"